# Defining a .geo file

In this section we discuss how to create a geometry file ($\texttt{.geo}$), with the geometry file being interpreted by Netgen (the meshing tool in NGSolve) we first explain how to create shapes and geometries using a $\texttt{.geo}$ file.

Due to Netgen being the underlying interpreter of the $\texttt{.geo}$ file we direct the user to the relevant Constructive Solid Geometry (CSG) documentation available at [http://netgen-mesher.sourceforge.net/docs/ng4.pdf](http://netgen-mesher.sourceforge.net/docs/ng4.pdf) and the large number of example geo files avaliable on the Netgen github page ([https://github.com/NGSolve/netgen/tree/master/tutorials](https://github.com/NGSolve/netgen/tree/master/tutorials)).

## Controlling the material parameters f a conducting sphere using a .geo file

So far we haved considered the MPT characterisation of a conducting sphere with $\sigma=10^6$ S/m,  and radius $\alpha=0.001$ m by constructing a coarse unstructured tetrahedral discretisation of a unit sphere set in truncated unbounded region in the form of a larger sphere of radius 100 units using order $p=2$ elements. Note the physical object is obtained by scaling the non-dimensional computational domain by $\alpha$. Now wish to control the materials of the sphere in more detail.

In the folder $\texttt{Geofiles/}$ there are a series of pre-setup object descriptions including $\texttt{sphere.geo}$ that contains the following

```
algebraic3d

solid sphout = sphere (0, 0, 0; 100);
solid sphin = sphere (0, 0, 0; 1) -maxh=0.5;

solid rest = sphout and not sphin;

tlo rest -transparent -col=[0,0,1];#air
tlo sphin -col=[1,0,0];#sphere -mur=10 -sig=1e6

```

All $\texttt{.geo}$ files follow the same format and will not work otherwise.

```
algebraic3d
```
is the top level command that every .geo file requires before any non commented lines. Comments are denoted using $\texttt{#}$ and continue to the end of the line.

named components of the $\texttt{.geo}$ file are identified using the keyword $\texttt{solid}$, and constructed using object primatives. For example
```
solid sphout = sphere (0, 0, 0; 100);

solid sphin = sphere (0, 0, 0; 1) -maxh=0.5;
```
defines two spheres, $\texttt{sphout}$ and $\texttt{sphin}$, centered at the origin and of radii 100 and 1 respectivly that we can reuse later. 
The $\texttt{-maxh}$ tag can be used to directly specify the maximum element size inside the associated object. For example, here we set the maximum element size inside the inner sphere to be 0.5. <b> the $\texttt{-maxh}$ tag overrides the mesh size option in $\texttt{MPT-Calculator}$.</b>

Complex objects are constructed by Eulerian operations applied to primatives. The keywords $\texttt{and}$ and $\texttt{or}$ denote the intersection and union respectivlty of the two solids, so the line
```
solid rest = sphout and not sphin;
```
defines a solid that contains only the region that is occupied by the outer sphere and not the inner sphere.

Finally, we wish to define which objects will be meshed. This is done using the $\texttt{tlo}$ tag.
```
tlo rest -transparent -col=[0,0,1];#air
tlo sphin -col=[1,0,0];#sphere -mur=10 -sig=1e6
```
The object names, $\texttt{-mur}$, and $\texttt{-sig}$ are used by MPT-Calculator to assign material properites to each object and are not used by the mesher.

The $\texttt{#air}$ is used to denote vacuum permeability and conductivity, thus this example defines a small sphere of $\mu_r = 1$ and $\sigma=10^6$ S/m surrounded by a larger non-conductive non-magnetic sphere.

## Examples of different material properties
In this section, we explore the effects of altering the material properties in the .geo file.

We begin by importing the required packages and naviagating to the project root directory

In [2]:
from time import time
import numpy as np
from matplotlib import pyplot as plt
import os
root_dir = os.path.dirname(os.getcwd())
os.chdir(root_dir)
print(os.getcwd())

C:\Users\James\Desktop\Updated_MPT_Calculator


The user is encouraged to make changes to the material properties of the $\texttt{sphere.geo}$ file and run main()

In [ ]:
from runner import main
main()

We see that making changes to $\sigma$ corresponds to a translation in the MPT spectral signatue, wheras changing $\mu_r$ alters the shape and magnitude of the curves.

Also note that each different version of $\texttt{sphere.geo}$ is saved under a different subdirectory in the Results folder.


## Earring

For a more complex object, we work in a very similar way. i.e. defining object primatives and constructing other shapes via unions and intersections.
In this example, we build a simple earring, consisting of spherical end, a cylindrical post, and a flat cylindric back.

A sphere is defined by an origin and a radius r
```
sphere(x, y, z; r)
```

A cylinder in the .geo format is assumed to be infinitly long. To construct a finite cylinder, we need to calculate the intersection with a plane. First to define a cylinder we can use the cylinder primative, which is defined by two points on the central axis and a radius r
```
cylinder(ax,ay,az; bx,by,bx; r)
```

and finally, to define a plane, we require a point on the plane, and a normal vector
```
plane(px,py,pz; nx, ny, nz)
```
A complete description of the other object primatives, as well as defining spline curves and surfaces is provided in the full documentation [http://netgen-mesher.sourceforge.net/docs/ng4.pdf](http://netgen-mesher.sourceforge.net/docs/ng4.pdf)

Combining these primatives produces the $\texttt{.geo} file description of the earring:
```
algebraic3d

#Outer domain
solid outer_box = orthobrick (-1000, -1000, -1000; 1000, 1000, 1000);

#Object parts
solid ball = sphere(6,0,0;1);
solid bar = cylinder(0,0,0;10,0,0;0.5)
	and plane(0,0,0;-1,0,0)
	and plane(6,0,0;1,0,0);
solid end = cylinder(-1,0,0;1,0,0;1.2)
	and plane(0,0,0;1,0,0)
	and plane(-0.5,0,0;-1,0,0);

#Combining parts
solid earing = ball or bar or end -maxh=0.25;
solid rest = outer_box and not earing;

#Defining top level objects
tlo rest -transparent -col=[0,0,1];#air
tlo earing -col=[1,0,0];#earring -mur=1 -sig=4.1E+07
```

## Specifying a .geo file to use with MPT-Calculator

We can specify a .geo file to use when running $\texttt{MPT-Calculator}$ via the $\texttt{geometry}$ argument:
```python
main(geometry='sphere.geo')
```
runs the sphere example we have been working with thus far wheras calling
```python
main(geometry='earring.geo')
```
runs the same sweep settings, but for the object defined in $\texttt{earring.geo}$

In [ ]:
main(geometry='earring.geo')

## Example .geo files
In the $\texttt{Geofiles/}$ directory there are many simple examples including object primatives such as a cylinder or cuboids. More complex examples include a rifle shell ($\texttt{rifle.geo}$), different types of large knifes ($\texttt{Knife_Cheap_Chef.geo}$, $\texttt{Knife_Cheap_Cutlet.geo}$), and an irregular tetrahedron ($\texttt{Tetra.geo}$).

Further examples can be found on the Netgen github page: [https://github.com/NGSolve/netgen/tree/master/tutorials](https://github.com/NGSolve/netgen/tree/master/tutorials)
